# Analyze the usage of the Tool

In [ ]:
# all other requirements will be installed with the application
%pip install -q pandas matplotlib seaborn user-agents

In [110]:
import sqlite3
import pandas as pd             #db querys with visualization
import matplotlib.pyplot as plt # diagrams
import matplotlib.image as mpimg # images
import seaborn as sns           # Heatmaps etc.
from user_agents import parse   # Split OS. Browser etc.
import math                     #  calc sizes
import config

## Open Database Connection

In [12]:
# check that the path is fitting to your config file
# in theory we could also use the config.py to load this information (maybe next version)
connection = sqlite3.connect("analytics.db")
cursor = connection.cursor()

In [ ]:
def showBar(df,title, x_column,y_column,x_label=None,y_label=None, show_x_values=True):
    """generates a bar chart in Jupyter Notebook"""
    if not x_label: x_label=x_column
    if not y_label: _ylabel=y_column
    ax = df.plot.bar(x=x_column, y=y_column, color="skyblue", legend=show_x_values)    
    for container in ax.containers:
        ax.bar_label(container, fmt='%d', label_type='edge') 
        if show_x_values: plt.xlabel=x_label
        if not show_x_values: ax.set_xticks([]) 
        plt.ylabel=y_label
        plt.title(title)
        # optimization depending on your data
        #plt.figure(figsize=(10,6))
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def showImage(path: str, name: str = ""):
    """shows an image in Jupyter Notebook"""
    img = mpimg.imread(path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image {name}")
    plt.show()

def showImageGrid(df: pd.DataFrame, path_column: str, name_column: str = None, descr_column: str = None):
    n_cols = 3
    n_rows = math.ceil(len(df)/n_cols)

    # Calculate plot size
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 10))

    # load images and text
    for idx, (ax, row) in enumerate(zip(axes.flatten(), df.iterrows())):
        _, row_data = row
        try:
            img = mpimg.imread(row_data[path_column])
            
            ax.imshow(img)
            ax.axis('off')  
            if name_column:
                title = row_data[name_column]
                if descr_column:
                    title = f"{title} \n {row_data[descr_column]}"
                ax.set_title(title, fontsize=12)
        except FileNotFoundError:
            ax.text(0.5, 0.5, 'Image not found', fontsize=12, ha='center', va='center')
            ax.axis('off')  

    for ax in axes.flatten()[len(df):]:
        ax.axis('off')  # Entferne leere Zellen

    # Align space between images
    plt.tight_layout()
    plt.show()

config.read_configuration()
# def get_os(user_agent):
#     os = (parse(user_agent)).os
#     if (os.version_string!=""):
#         return os.family + " " + os.version_string
#     else:
#         return os.family
# def enhance_data(df_sessions):
#     # function to put make the dataframe more feature rich and simplify queries later
#     df_sessions['Datetime'] = pd.to_datetime(df_sessions["Timestamp"])
#     # For analysis it is interesting which day of week has the entry
#     df_sessions['Day'] = df_sessions["Datetime"].dt.day_name()
#     df_sessions['OS'] = df_sessions["Client"].apply(get_os) #alternate way is lambda, but a function can be more complex
#     df_sessions['Browser'] = df_sessions["Client"].apply(lambda ua: parse(ua).browser.family)
#     df_sessions['IsMobile'] = df_sessions["Client"].apply(lambda ua: parse(ua).is_mobile) 
#     df_sessions['IsBot'] = df_sessions["Client"].apply(lambda ua: parse(ua).is_bot)
#     return df_sessions

## Analyze sources of the Queries

### read and prepare Data for analyzation
This section must be executed for all analyzation regading the Loaction, Operating Systems, and usage time

Here we first read all Data into the Memory and analyze it then. It's more performant then adHoc Queries but AdHoc might be better if the Server is running in parallel.

In [ ]:
query = "select * from tblSessions order by timestamp"
df_sessions = enhance_data(pd.read_sql_query(query, connection))
df_sessions.tail()

**Filter by date (optional!)**
if it is useful for your analysis

In [ ]:
# # Year-Month-Day Hours:Minutes:Seconds
# start_date = "2025-01-01 00:00:00"
# end_date = "2025-12-31 23:59:59"
# query = f"select * from tblSessions where Timestamp between '{start_date}' and '{end_date}'"
# df_sessions = pd.read_sql_query(query, connection)
# df_sessions = enhance_data(df_sessions)
# df_sessions.tail()

### Where does the Users comes from

#### Prepare Dataset for this section

Hint: it's required to execute the "read an prepare Data" first!



In [ ]:
df_grouped_location = df_sessions.groupby(["Continent", "Country", "City"]).size().reset_index(name="SessionCount")
#remove correct NaN for better work with Panda groups
df_grouped_location["Continent"] = df_grouped_location["Continent"].fillna("unkonwn")
df_grouped_location["Country"] = df_grouped_location["Country"].fillna("unkonwn")
df_grouped_location["City"] = df_grouped_location["City"].fillna("unkonwn")
df_grouped_location.head(10)

Sample of AdHoc Query for the same content as below.
Use it only if you need special data or rrlations or if your amount of data is very big

In [ ]:
query = "select Continent, Country, City, count(1) as SessionCount from tblSessions group by Continent, Country, City"
df_where = pd.read_sql_query(query, connection)
df_where.head(10)

In [ ]:
#Group by Countries
df_country = df_grouped_location.groupby("Country")["SessionCount"].sum().reset_index()
df_country = df_country.sort_values(by="SessionCount", ascending=False)
plt.bar(df_country["Country"], df_country["SessionCount"], color="skyblue")
plt.xlabel="Country"
plt.ylabel="Number of Sessions"
plt.title("Number of Sessions by Country")
# optimization depending on your data
#plt.figure(figsize=(10,6))
plt.xticks(rotation=90)
plt.show()

Select a country to get more details about the cities

In [ ]:
selected_country = "The Netherlands"
df_citys = df_grouped_location[df_grouped_location["Country"]==selected_country]
# sort by count
df_citys = df_citys.sort_values(by="SessionCount", ascending=False)
plt.bar(df_citys["City"], df_citys["SessionCount"])
plt.xlabel="City"
plt.ylabel="Number of Sessions"
plt.title(f"Numer of Sessions by City in {selected_country}")
plt.show()

### Which time accesses the users the system?

In [ ]:
query = """
select strftime('%Y-%m-%d', Timestamp) as Date, Count(*) as SessionCount
from tblSessions
Group by Date
Order by Date
"""

df_time_distribution = pd.read_sql_query(query, connection)
df_time_distribution.head()

Day of Week and Time

In [ ]:
df_time_distribution = df_sessions
df_time_distribution['Hour'] = df_time_distribution["Datetime"].dt.hour
df_time_distribution = df_sessions.groupby(["Day", "Hour"]).size().reset_index(name="SessionCount")
df_time_distribution.head()

In [ ]:
heatmap = df_time_distribution.pivot_table(index="Day", columns="Hour", values="SessionCount", aggfunc="sum", fill_value=0)
ordered_days= ["Monday", "Tuesday","Wednesday","Thursday", "Friday","Saturday","Sunday"]
heatmap = heatmap.reindex(ordered_days)
sns.heatmap(heatmap, cmap="coolwarm", annot=True, fmt="g")
plt.title("Heatmap of Sessions by Hour and Weekday")
plt.xlabel = "Hour of Day"
plt.ylabel = "Day of Week"
plt.tight_layout()
plt.show()

### Browser, Languages und Operating Systems

In [ ]:
df = df_sessions.groupby("OS").size().reset_index(name="SessionCount")
df = df.sort_values(by="SessionCount", ascending=False)
showBar(df, "Sessions by OS", "OS", "SessionCount")

How many Mobile Devices

In [ ]:
counts = df_sessions['IsMobile'].value_counts()
print(counts)
counts.plot.pie(
    labels=['Desktop', 'Mobile'],  #  1 = Mobile, 0 = Desktop
    autopct='%1.1f%%',  # Format (Percentage)
    startangle=90,  # Start angel
    colors=['lightblue', 'lightgreen']  
)
plt.title("How many users using a mobile device")
plt.ylabel=""
plt.xlabel=""
plt.show()

Analyze by Languages (important if location is not available)

In [ ]:
df = df_sessions.groupby("Languages").size().reset_index(name="SessionCount")
df = df.sort_values(by="SessionCount", ascending=False)
showBar(df,
        x_column="Languages",
        y_column="SessionCount",
        title="Number of Sessions by Languages")


## Analyze the Images

In [ ]:
query = "select * from tblGenerations"
df_generations = pd.read_sql_query(query, connection)
print(f"{len(df_generations)} generations total")

In [ ]:
# latestes entries
df_generations.tail()

Which Style was used

In [ ]:
df = df_generations.groupby("Style").size().reset_index(name="SessionCount")
df = df.sort_values(by="SessionCount", ascending=False)
showBar(df,
        x_column="Style",
        y_column="SessionCount",
        title="Usage of Styles")


Average generations per input image

In [ ]:
df = df_generations.groupby("Input_SHA1").size().reset_index(name="SessionCount")
average = df['SessionCount'].mean()
print(f"Average generation count per Input is {math.ceil(average)}")
df = df.sort_values(by="SessionCount", ascending=False)
showBar(df,
        x_column="Input_SHA1",
        y_column="SessionCount",
        title="Generations per Image",
        show_x_values=False
        )

Top 5 Source images with amount of generations

In [ ]:
df = df_generations.groupby("Input_SHA1").size().reset_index(name="SessionCount")
df = df.sort_values(by="SessionCount", ascending=False)
df = df.head(5)
df['SourcePath'] = df["Input_SHA1"].apply(lambda sha: f"{config.get_cache_folder()}{sha}.jpg")
showImageGrid(df,"SourcePath",name_column="SessionCount")
df.head(len(df))

### Analyze a dedicated Image or generation Session

Find prompts used for an image and relevant images.

**TASK**: Put the filename without extension into SHA1

In [ ]:
#just copy from filename (also output filename can be used)
SHA1 = ""
query = f"select Session,Style,Userprompt,output as path from tblGenerations where input_sha1='{SHA1}' or output like '%{SHA1}%'"
# todo: use df_generations and filter then 
df = pd.read_sql_query(query, connection)

showImageGrid(
    df=df,
    path_column="path",
    name_column="Style",
    descr_column="Userprompt"
    )
df.tail()

Find details to the User by using Session 

**TASK**: copy session from output above and add to "Session" variable

In [ ]:
#Add here the Session from above
Session = ""

# don't change this
query = f"select OS, Browser, Language from tblSessions where session={Session}"
# todo: use df_generations and filter then 
df = pd.read_sql_query(query, connection)
df.head()

All images uploaded by same author (Session)

**TASK**: copy session from output above and add to "Session" variable

In [ ]:
#Add here the Session from above
#Session = ""

# don't change this
query = f"select distinct Input_SHA1 from tblGenerations where session='{Session}'"

df = pd.read_sql_query(query, connection)
df['SourcePath'] = df["Input_SHA1"].apply(lambda sha: f"{config.get_cache_folder()}{sha}.jpg")
showImageGrid(df,"SourcePath")
df.head(15)


Show all generated image of this user

**TASK**: copy session from output above and add to "Session" variable

In [ ]:
Session = ""

# don't change this
# show all generated images of this user
query = f"select Timestamp, Input_SHA1 as Image, Style, Userprompt,Output as path from tblGenerations where session='{Session}' "
# todo: use df_generations and filter from df_details above like df_details['Session'] then 
df = pd.read_sql_query(query, connection)
showImageGrid(
    df=df,
    path_column="path",
    name_column="Style",
    descr_column="Userprompt"
    )
df.head(len(df))
